# 1.1. Scrapers estaciones meteorológicas

La siguiente celda importa varios módulos y bibliotecas de Python necesarios para llevar a cabo tareas de web scraping y manejo de datos. En resumen:

- `selenium`: una biblioteca de Python para automatizar la navegación web.
- `pandas`: una biblioteca de Python para la manipulación de datos.
- `unicodedata`: una biblioteca de Python para trabajar con caracteres Unicode.
- `logging`: una biblioteca de Python para la generación de registros (logs).
- `time`: una biblioteca de Python para el manejo del tiempo.
- `os`: una biblioteca de Python para interactuar con el sistema operativo.

Además, se define una excepción personalizada para detectar cuándo un elemento no existe en la página web (`NoSuchElementException`), se configura el driver de Chrome para que se descargue automáticamente (`ChromeDriverManager`), se establecen opciones de configuración para el driver de Chrome (`Options`), se importa el servicio para ejecutar el driver de Chrome (`Service`) y se define el formato de fecha y hora utilizado en los registros (`datetime`).


In [1]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
from datetime import datetime

import pandas as pd
import unicodedata
import logging
import time
import os

La siguiente función `log_file_name()` retorna una cadena de texto que representa la ruta del archivo de registro (log) de la información producida por el proceso de scraping. La ruta es construida utilizando el módulo os para acceder a la variable de entorno HOME del sistema operativo y se concatenan diferentes partes de la ruta para crear la ubicación del archivo de registro. El nombre del archivo incluye la fecha y hora actual en el formato "YYYY-MM-DD_HH:MM:SS" para identificar de forma única cada archivo generado.

In [2]:
def log_file_name():
    timestamp = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
    return f"{os.getenv('HOME')}/Escritorio/Tesina/Analisis_4.0/datos_clima/logs/scraping-sipas-{timestamp}.log"

La función `pasar_a_texto_elemento` toma como entrada un objeto `elemento` y utiliza el método `get_attribute` para obtener el texto contenido en el atributo `innerText`. A continuación, utiliza la función `unicodedata.normalize` con el parámetro `'NFKC'` para normalizar el texto, es decir, para eliminar cualquier carácter especial o símbolo que pueda aparecer en el texto, como espacios en blanco adicionales o caracteres unicode no reconocidos. Finalmente, devuelve el texto normalizado como una cadena de caracteres.

In [3]:
def pasar_a_texto_elemento(elemento):
    elemento_texto = elemento.get_attribute('innerText')    
    return unicodedata.normalize('NFKC',elemento_texto)

La función `configurar_web_driver` configura el objeto webdriver.Chrome para usarlo en la automatización del navegador web Google Chrome.

Primero, se crea un objeto `Options` para definir las opciones de configuración de Chrome. En este caso, se establece `options.headless = True` para ejecutar el navegador en modo sin cabeza (headless), es decir, sin abrir la interfaz gráfica del navegador, y se establece la dimensión de la ventana con `options.add_argument("--window-size=1920,1200")`.

Luego, se define un objeto `handlers` que contiene un `logging.FileHandler` para escribir los mensajes de registro en un archivo de registro en disco y un `logging.StreamHandler` para mostrar los mensajes en la consola.

Por último, se llama al constructor de `webdriver.Chrome`, se instala el driver de Chrome mediante `ChromeDriverManager().install()`, se pasan las opciones definidas anteriormente y se retorna el objeto `webdriver.Chrome` configurado.

In [4]:
def configurar_web_driver():
    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")
    handlers = [logging.FileHandler(log_file_name()), logging.StreamHandler()]
    logging.basicConfig(
        format='%(asctime)s - %(message)s',
        level=logging.INFO, handlers=handlers
    )
    return webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), 
        options=options
    )

Existen 60 estaciones sin embargo cada sonda tenía una ID que no necesariamente se correspondía con el número de sondas disponibles, lo que resultó en algunas sondas con números de ID mayores al total de 60 sondas. Para abordar este problema, se recorrió un rango de números que iba del 1 al 72 para recopilar los datos de cada una de las 60 sondas disponibles.

In [5]:
numero_estaciones = 71 # De 1-71 

La función `obtener_datos_para_url` recibe un objeto `driver` y una URL. Luego, itera sobre una serie de números de estación y reemplaza el identificador "idNum" en la URL con el número de estación actual para acceder a la página web correspondiente a cada estación meteorológica.

Una vez en la página web, espera hasta que el elemento de la tabla esté cargado y luego obtiene la cabecera de la tabla si aún no se ha obtenido. Luego, busca la estación meteorológica y los datos correspondientes en la tabla y los agrega a un DataFrame que contiene los datos para todas las estaciones.

La función utiliza la librería `logging` para imprimir información sobre el progreso de la iteración. Además, utiliza la función `pasar_a_texto_elemento` para obtener el texto de cada elemento HTML y normalizarlo en Unicode.

In [12]:
def obtener_datos_para_url(driver, url):
    cabecera_df=['Estación']
    datos_df=[]
    cantidad_elementos_cabecera = 15
    for numero_estacion in range(1,numero_estaciones):
        logging.info(f"Número estación: {numero_estacion}")
        url_estacion = url['url'].replace('idNum', str(numero_estacion))
        driver.get(url_estacion)
        time.sleep(120)
        if(numero_estacion==1):
            cabecera = driver.find_element(By.XPATH,'//*[@id="Contenido"]/div/table/thead/tr')
            elementos_cabecera = cabecera.find_elements(By.XPATH, './/th')
            for elemento in elementos_cabecera:
                cabecera_df.append(pasar_a_texto_elemento(elemento))
        try:
            cuerpo = driver.find_element(By.XPATH, '//*[@id="Contenido"]/div/table/tbody[2]')
            estacion = pasar_a_texto_elemento(driver.find_element(By.XPATH, '//*[@id="Contenido"]/p/a[2]/span'))
            estacion = estacion.replace('Estación meteorologica: ','')
            logging.info(f"Estación: {estacion}")
        except NoSuchElementException:
            logging.info(f"No hay datos")
            continue
        anios_elementos = cuerpo.find_elements(By.XPATH, './/tr')
        for anio_elemento in anios_elementos:
            logging.info(f"Entre/Cambie de año")
            datos_anio = [estacion]
            for indice in range(1,cantidad_elementos_cabecera):
                datos_anio.append(pasar_a_texto_elemento(anio_elemento.find_element(By.XPATH, f".//td[{indice}]")))
            datos_df.append(datos_anio)
        time.sleep(30)

    return pd.DataFrame(datos_df, columns=cabecera_df)

Este código define una lista llamada `urls` que contiene cinco diccionarios. Cada diccionario contiene dos claves: `'url'` y `'filename'`. La clave `'url'` contiene una URL que se utilizará más adelante en el código para obtener datos meteorológicos de una estación específica. La clave `'filename'` contiene el nombre de archivo que se utilizará para guardar los datos obtenidos de esa URL. 

In [14]:
urls = [
{
'url': 'https://sipas.inta.gob.ar/?q=agrometeorologia-estadist-temp-min&idEstacion=idNum',
'filename': 'temperatura_minima_estaciones.csv'},
{
'url': 'https://sipas.inta.gob.ar/?q=agrometeorologia-estadist-temp-med&idEstacion=idNum',
'filename': 'temperatura_media_estaciones.csv'},
{
'url': 'https://sipas.inta.gob.ar/?q=agrometeorologia-estadist-temp-max&idEstacion=idNum',
'filename': 'temperatura_maxima_estaciones.csv'},
{
'url': 'https://sipas.inta.gob.ar/?q=agrometeorologia-estadist-hum&idEstacion=idNum',
'filename': 'humedad_estaciones.csv'},
{
'url': 'https://sipas.inta.gob.ar/?q=agrometeorologia-estadist-lluvia&idEstacion=idNum', 
'filename': 'lluvia_estaciones.csv'
}]

El siguiente código utiliza un webdriver para automatizar la descarga de datos de varias estaciones meteorológicas y guardar los datos en archivos CSV.

En primer lugar, se llama a la función `configurar_web_driver()` para configurar el webdriver. Luego, se itera sobre una lista de diccionarios `urls`, donde cada elemento del diccionario tiene una URL y un nombre de archivo CSV asociado.

Para cada URL en la lista, se llama a la función `obtener_datos_para_url(driver, url)` para extraer los datos de la página web correspondiente utilizando el webdriver. Los datos se almacenan en un DataFrame de pandas y luego se guardan en un archivo CSV con el nombre especificado en el diccionario.

Después de procesar todas las URL en la lista, el webdriver se cierra llamando a `driver.quit()`.

In [13]:
driver = configurar_web_driver()

for url in urls:
    df_final = obtener_datos_para_url(driver, url)
    df_final.to_csv(url['filename'], index=None)

driver.quit()

[WDM] - ====== WebDriver manager ======
2023-04-05 09:09:26,895 - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
2023-04-05 09:09:26,926 - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
2023-04-05 09:09:26,927 - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [/home/lucila/.wdm/drivers/chromedriver/linux64/111.0.5563.64/chromedriver] found in cache
2023-04-05 09:09:27,062 - Driver [/home/lucila/.wdm/drivers/chromedriver/linux64/111.0.5563.64/chromedriver] found in cache
2023-04-05 09:09:27,683 - Número estación: 1
2023-04-05 09:11:32,582 - Estación: Trelew
2023-04-05 09:11:32,590 - Entre/Cambie de año
2023-04-05 09:11:32,730 - Entre/Cambie de año
2023-04-05 09:11:32,871 - Entre/Cambie de año
2023-04-05 09:11:33,015 - Entre/Cambie de año
2023-04-05 09:11:33,157 - Entre/Cambie de año
2023-04-05 09:11:33,302 - Entre/Cambie de año
2023-04-05 09:11:33,449 - Entr